In [3]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [21]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [22]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood


In [23]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,'Borough': borough,'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,M2A\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n
...,...,...,...
130,M9Y\n,Not assigned\n,M1Z\n
131,M2Z\n,Not assigned\n,M3Z\n
132,M4Z\n,Not assigned\n,M5Z\n
133,M6Z\n,Not assigned\n,M7Z\n


In [11]:
neighbors.shape

(135, 3)

In [ ]:
df_geo_coordinate = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo_coordinate.head()

In [10]:
df_geo_coordinate = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_geo_coordinate.shape

(103, 3)

In [13]:
df_geo_coordinate.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_geo_coordinate.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# merge
neighbors.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,M2A\n
1,M3A\n,North York\n,Parkwoods\n
2,M4A\n,North York\n,Victoria Village\n
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n


In [16]:
df_geo_coordinate.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_merged = neighbors.join(df_geo_coordinate.set_index('PostalCode'), on='PostalCode')
df_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,M2A\n,NaN,NaN
1,M3A\n,North York\n,Parkwoods\n,NaN,NaN
2,M4A\n,North York\n,Victoria Village\n,NaN,NaN
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n,NaN,NaN
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n,NaN,NaN
...,...,...,...,...,...
130,M9Y\n,Not assigned\n,M1Z\n,NaN,NaN
131,M2Z\n,Not assigned\n,M3Z\n,NaN,NaN
132,M4Z\n,Not assigned\n,M5Z\n,NaN,NaN
133,M6Z\n,Not assigned\n,M7Z\n,NaN,NaN
